# 6강) FAISS 실습

### Requirements

In [1]:
# !pip install datasets
# !pip install transformers
# !pip install faiss-cpu

## 5강의 자료를 활용해 passage / question enocoder 학습


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np
from tqdm import tqdm, trange
import random
import torch
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
 
torch.manual_seed(2021)
torch.cuda.manual_seed(2021)

In [3]:
dataset = load_dataset("squad_kor_v1")
corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus)
 
model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Reusing dataset squad_kor_v1 (/opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7)


Training Dataset 준비

In [4]:
# Use subset (128 examples) of original training dataset 
sample_idx = np.random.choice(range(len(dataset['train'])), 128)
training_dataset = dataset['train'][sample_idx]

q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
p_seqs = tokenizer(training_dataset['context'], padding="max_length", truncation=True, return_tensors='pt')

train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'], 
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])

BERT encoder 학습시키기

In [5]:
class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)
 
    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)
      
      pooled_output = outputs[1]
 
      return pooled_output
 
 
def train(args, dataset, p_model, q_model):
  
  # Dataloader
  train_sampler = RandomSampler(dataset)
  train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)
 
  # Optimizer
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
        {'params': [p for n, p in p_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in p_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in q_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in q_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
  t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)
 
  # Start training!
  global_step = 0
  
  p_model.zero_grad()
  q_model.zero_grad()
  torch.cuda.empty_cache()
  
  train_iterator = trange(int(args.num_train_epochs), desc="Epoch")
 
  for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")
 
    for step, batch in enumerate(epoch_iterator):
      q_encoder.train()
      p_encoder.train()
      
      if torch.cuda.is_available():
        batch = tuple(t.cuda() for t in batch)
 
      p_inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2]
                  }
      
      q_inputs = {'input_ids': batch[3],
                  'attention_mask': batch[4],
                  'token_type_ids': batch[5]}
      
      p_outputs = p_model(**p_inputs)  # (batch_size, emb_dim)
      q_outputs = q_model(**q_inputs)  # (batch_size, emb_dim)
 
 
      # Calculate similarity score & loss
      sim_scores = torch.matmul(q_outputs, torch.transpose(p_outputs, 0, 1))  # (batch_size, emb_dim) x (emb_dim, batch_size) = (batch_size, batch_size)
 
      # target: position of positive samples = diagonal element 
      targets = torch.arange(0, args.per_device_train_batch_size).long()
      if torch.cuda.is_available():
        targets = targets.to('cuda')
 
      sim_scores = F.log_softmax(sim_scores, dim=1)
 
      loss = F.nll_loss(sim_scores, targets)
      print(loss)
 
      loss.backward()
      optimizer.step()
      scheduler.step()
      q_model.zero_grad()
      p_model.zero_grad()
      global_step += 1
      
      torch.cuda.empty_cache()
 
 
    
  return p_model, q_model

In [6]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01
)


In [7]:
# load pre-trained model on cuda (if available)
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.bias',

Train function 정의 후, 두개의 encoder fine-tuning 하기


In [8]:
p_encoder, q_encoder = train(args, train_dataset, p_encoder, q_encoder)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

tensor(15.6777, device='cuda:0', grad_fn=<NllLossBackward>)


tensor(21.0619, device='cuda:0', grad_fn=<NllLossBackward>)


tensor(15.4835, device='cuda:0', grad_fn=<NllLossBackward>)


tensor(16.6192, device='cuda:0', grad_fn=<NllLossBackward>)


...

tensor(1.3120, device='cuda:0', grad_fn=<NllLossBackward>)


tensor(1.4990, device='cuda:0', grad_fn=<NllLossBackward>)


tensor(2.8421, device='cuda:0', grad_fn=<NllLossBackward>)


tensor(1.5728, device='cuda:0', grad_fn=<NllLossBackward>)


Epoch: 100%|██████████| 2/2 [00:28<00:00, 14.42s/it]


## Passage retrieval 준비하기

Search corpus: KorQuAD validation context

In [9]:
search_corpus = list(set([example['context'] for example in dataset['validation']]))
len(search_corpus)

960

Passage encoder를 활용하여 passage dense embedding 생성

In [10]:
eval_batch_size = 8

# Construt dataloader
valid_p_seqs = tokenizer(search_corpus, padding="max_length", truncation=True, return_tensors='pt')
valid_dataset = TensorDataset(valid_p_seqs['input_ids'], valid_p_seqs['attention_mask'], valid_p_seqs['token_type_ids'])
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=eval_batch_size)

# Inference using the passage encoder to get dense embeddeings
p_embs = []

with torch.no_grad():

  epoch_iterator = tqdm(valid_dataloader, desc="Iteration", position=0, leave=True)
  p_encoder.eval()

  for _, batch in enumerate(epoch_iterator):
    batch = tuple(t.cuda() for t in batch)

    p_inputs = {'input_ids': batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2]
                }
        
    outputs = p_encoder(**p_inputs).to('cpu').numpy()
    p_embs.extend(outputs)

p_embs = np.array(p_embs)
p_embs.shape  # (num_passage, emb_dim)


Iteration: 100%|██████████| 120/120 [00:10<00:00, 11.98it/s]


(960, 768)

Question encoder를 활용해여 quesntion dense embedding 생성

In [11]:
np.random.seed(1)

sample_idx = np.random.choice(range(len(dataset['validation'])), 5)
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

query

['대한민국 제14대 대통령으로 향년 89세를 일기로 서거한 김영삼 대통령의 묘소가 있는 곳은?',
 '금강산의 겨울 이름은?',
 '유관순 열사는 당시 어떤 종교를 믿고 있었는가?',
 '1997년 10월 23일, 국회 본회의 대표 연설에서 전두환, 노태우 전 대통령에 대한 사면을 촉구한 새정치 국민회의 의원은?',
 '셰르징거가 찾아왔다가 우연히 푸시캣 돌스에 영입된 곳은?']

In [12]:
valid_q_seqs = tokenizer(query, padding="max_length", truncation=True, return_tensors='pt').to('cuda')

with torch.no_grad():
  q_encoder.eval()
  q_embs = q_encoder(**valid_q_seqs).to('cpu').numpy()

torch.cuda.empty_cache()

q_embs.shape  # (num_query, emb_dim)

(5, 768)

## GPU를 활용하여 passage retrieval 수행하기

GPU에서 exhaustive search 수행

In [13]:
if torch.cuda.is_available():
  p_embs_cuda = torch.Tensor(p_embs).to('cuda')
  q_embs_cuda = torch.Tensor(q_embs).to('cuda')


In [14]:
import time
start_time = time.time()

dot_prod_scores = torch.matmul(q_embs_cuda, torch.transpose(p_embs_cuda, 0, 1))

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(rank)

print("--- %s seconds ---" % (time.time() - start_time))

tensor([[231, 252, 268,  ..., 931, 938, 390],
        [231, 252, 946,  ..., 196, 390, 876],
        [404, 438, 515,  ...,  40, 898, 587],
        [785, 621, 200,  ..., 956, 469, 144],
        [231, 411, 301,  ..., 545, 390, 938]], device='cuda:0')
--- 0.0023641586303710938 seconds ---


In [15]:
k = 5 

for i, q in enumerate(query[:1]):
  print("[Search query]\n", q, "\n")
  print("[Ground truth passage]")
  print(ground_truth[i], "\n")

  r = rank[i]
  for j in range(k):
    print("Top-%d passage with score %.4f" % (j+1, dot_prod_scores[i][r[j]]))
    print(search_corpus[r[j]])
  print('\n')

[Search query]
 대한민국 제14대 대통령으로 향년 89세를 일기로 서거한 김영삼 대통령의 묘소가 있는 곳은? 

[Ground truth passage]
2015년 11월 10일 건강검진 차 서울대학교 병원을 찾아 17일까지 입원한 뒤 퇴원했다. 그러다, 이틀 뒤인 19일 고열과 혈액감염 의심 증세로 서울대학교 병원에 다시 입원한 후, 11월 21일 오후에 증세가 급격히 악화됨에 따라 중환자실로 옮겨졌다. 상태가 전혀 호전되지 않던 김영삼은 결국 2015년 11월 22일 오전 0시 21분 32초에 병마를 물리치지 못하고 혈액 감염 의심으로 치료를 받던 중 향년 89세의 일기로 서거하였다. 사망에 이른 직접적인 원인은 허약한 전신 상태에 패혈증과 급성 심부전이 겹쳐 일어난 것으로 판단되었다. 장례는 대한민국 최초로 5일간 국가장으로 치뤄졌다. 이는 국장과 국민장이 통합된 이후 처음 치뤄지는 국가장이다. 이어 11월 26일 국회의사당에서 영결식이 있었고 국립서울현충원에 안장되었다. 묘소의 정확한 위치는 제3장군묘역 우측능선에 위치하고 있으며 단독 묘역이다. 

Top-1 passage with score 23.2187
2011년 10월 15일, 10.26 서울시장 보궐선거에 나선 한나라당 나경원 후보가 자신의 트위터에서 스스로를 지지하는 ‘자화자찬’하는 글을 올려 논란이 일었다. 자신이 작성한 글을 리트윗(재인용)해 “콘텐츠가 있는 공약과 정책 정말 멋집니다” 등 자신을 지지하는 댓글을 달았다. 이에 네티즌들이 ‘나르시즘 나경원’ ‘자화자찬도 유분수’ ‘알바의 실수인가’ 등의 지적을 하자 나경원 측은 2011년 10월 16일 해당 트위터 글을 삭제하고 “확인 결과 시스템 간에 충돌이 일어나 계정 연동 오류가 발생한 것으로 확인됐다”며 “현재 오류를 바로 잡았다”고 밝혔다. 그러나 2011년 10월 20일 한 트위터 사용자가 나경원 트위터 멘션 오류에 대해 트위터 본사에 문의한 내용을 온라인에 게시했는데 트위터 본사 답변에 따르면 “나 후보 측의 트위터

## FAISS를 활용하여 CPU에서 passage retrieval 수행하기




FAISS SQ8, IVF 를 활용해서 cpu에서 passage retrieval 실습해보기

In [16]:
import faiss

num_clusters = 16
niter = 5
k = 5

# 1. Clustering
emb_dim = p_embs.shape[-1]
index_flat = faiss.IndexFlatL2(emb_dim)

clus = faiss.Clustering(emb_dim, num_clusters)
clus.verbose = True
clus.niter = niter
clus.train(p_embs, index_flat)
centroids = faiss.vector_float_to_array(clus.centroids)
centroids = centroids.reshape(num_clusters, emb_dim)

quantizer = faiss.IndexFlatL2(emb_dim)
quantizer.add(centroids)


Clustering 960 points in 768D to 16 clusters, redo 1 times, 5 iterations
  Preprocessing in 0.00 s



In [17]:
# 2. SQ8 + IVF indexer (IndexIVFScalarQuantizer)
indexer = faiss.IndexIVFScalarQuantizer(quantizer, quantizer.d, quantizer.ntotal, faiss.METRIC_L2)
indexer.train(p_embs)
indexer.add(p_embs)

In [18]:
# 3. Search using indexer

start_time = time.time()
D, I = indexer.search(q_embs, k)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.001428365707397461 seconds ---


In [19]:
print('=======[Distance]=======')
print(D)
print('\n')
print('=======[Index of Top-5 Passages]=======')
print(I)

=======[Distance]=======
[[127.542694 129.06415  129.45184  130.21706  130.48953 ]
 [134.16019  137.2977   138.8204   139.31395  139.47934 ]
 [152.69353  154.42328  154.63792  155.64421  155.67175 ]
 [133.84999  135.56204  135.80368  136.24295  136.92299 ]
 [127.75181  129.40057  129.5321   130.41554  130.56493 ]]


=======[Index of Top-5 Passages]=======
[[112 245 799 158 823]
 [110 230  15 819 726]
 [112 245 799 823 158]
 [112 245 799 158 823]
 [112 245 799 158 823]]


In [20]:
for i, q in enumerate(query[:1]):
  print("[Search query]\n", q, "\n")
  print("[Ground truth passage]")
  print(ground_truth[i], "\n")

  d = D[i]
  i = I[i]
  for j in range(k):
    print("Top-%d passage with distance %.4f" % (j+1, d[j]))
    print(search_corpus[i[j]])
  print('\n')

[Search query]
 대한민국 제14대 대통령으로 향년 89세를 일기로 서거한 김영삼 대통령의 묘소가 있는 곳은? 

[Ground truth passage]
2015년 11월 10일 건강검진 차 서울대학교 병원을 찾아 17일까지 입원한 뒤 퇴원했다. 그러다, 이틀 뒤인 19일 고열과 혈액감염 의심 증세로 서울대학교 병원에 다시 입원한 후, 11월 21일 오후에 증세가 급격히 악화됨에 따라 중환자실로 옮겨졌다. 상태가 전혀 호전되지 않던 김영삼은 결국 2015년 11월 22일 오전 0시 21분 32초에 병마를 물리치지 못하고 혈액 감염 의심으로 치료를 받던 중 향년 89세의 일기로 서거하였다. 사망에 이른 직접적인 원인은 허약한 전신 상태에 패혈증과 급성 심부전이 겹쳐 일어난 것으로 판단되었다. 장례는 대한민국 최초로 5일간 국가장으로 치뤄졌다. 이는 국장과 국민장이 통합된 이후 처음 치뤄지는 국가장이다. 이어 11월 26일 국회의사당에서 영결식이 있었고 국립서울현충원에 안장되었다. 묘소의 정확한 위치는 제3장군묘역 우측능선에 위치하고 있으며 단독 묘역이다. 

Top-1 passage with distance 127.5427
감리교회는 일찍이 전래 당시 조선의 독립을 위해 노력한 교단이었다. 아펜젤러 선교사는 한국의 독립운동에 깊이 관여하여 독립협회 창설에 도움을 주었으며, 배재학당 출신 이승만을 옥중지원했으며, 3.1독립 선언을 한 33인중 이필주, 신석구 등 감리교인 9명은 장로교인 7명 함께 개신교 대표로 참여했다. 이후 일제 탄압을 받았고, 독립운동에도 직간접적으로 참여하였고, 유관순 열사는 감리교인이었다. 특히 스크랜턴 선교사는 전덕기를 전도하여 서울 숭례문(남대문)에 있던 상동감리교회에서 감리교청년 연합회인 엡웟회와 상동청년학원을 민족의식을 가진 청년들이 모여 을사조약 반대운동을 하였고, 독립 운동의 중추적 역할을 한 "신민회"는 감리교회의 지원으로 탄생하였다. 특히 신민회가 조직된 상동감리교회는 이준의 헤이그 특사 파견을 협의하고 실행한 